<a href="https://colab.research.google.com/github/Sankytanky100/Data_Science-Projects/blob/main/Analyze_Twitch_Gaming_Data_Using_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install ipython-sql extension
!pip install ipython-sql

# Install SQLAlchemy for SQLite support
!pip install sqlalchemy

# Load the SQL extension
%load_ext sql

from sqlalchemy import create_engine

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
# Create an in-memory SQLite database
%sql sqlite://


In [ ]:
import pandas as pd

# Read the CSV files into Pandas DataFrames
stream_df = pd.read_csv('video_play.csv')
chat_df = pd.read_csv('chat.csv')

# Remove any spaces in column names (if any)
stream_df.columns = [c.replace(' ', '_') for c in stream_df.columns]
chat_df.columns = [c.replace(' ', '_') for c in chat_df.columns]

# Load the DataFrames into SQLite
from sqlalchemy import create_engine

# Create a SQLite database file
engine = create_engine('sqlite:///twitch.db', echo=False)

# Load data into SQLite database
stream_df.to_sql('stream', con=engine, index=False, if_exists='replace')
chat_df.to_sql('chat', con=engine, index=False, if_exists='replace')

# Connect ipython-sql to the database file
%sql sqlite:///twitch.db


In [ ]:
%%sql
SELECT * FROM stream LIMIT 20;

SyntaxError: invalid syntax (<ipython-input-11-70e081a6c976>, line 2)

In [ ]:
%%sql
SELECT * FROM chat LIMIT 20;


In [ ]:
%%sql
SELECT DISTINCT game FROM stream;


In [ ]:
%%sql
SELECT DISTINCT channel FROM stream;


In [ ]:
%%sql
SELECT
    game,
    SUM(viewers) AS total_viewers
FROM stream
GROUP BY game
ORDER BY total_viewers DESC;


In [ ]:
%%sql
SELECT
    country,
    SUM(viewers) AS total_viewers
FROM stream
WHERE game = 'League of Legends'
GROUP BY country
ORDER BY total_viewers DESC;


In [ ]:
%%sql
SELECT
    player,
    COUNT(DISTINCT user_id) AS num_streamers
FROM stream
GROUP BY player
ORDER BY num_streamers DESC;


Create a New Column Named genre for Each of the Games
Group the games into their genres:

MOBA: 'League of Legends', 'Dota 2', 'Heroes of the Storm'
FPS: 'Counter-Strike: Global Offensive'
Survival: 'DayZ', 'ARK: Survival Evolved'
Other: All other games

In [ ]:
%%sql
SELECT
    game,
    CASE
        WHEN game IN ('League of Legends', 'Dota 2', 'Heroes of the Storm') THEN 'MOBA'
        WHEN game = 'Counter-Strike: Global Offensive' THEN 'FPS'
        WHEN game IN ('DayZ', 'ARK: Survival Evolved') THEN 'Survival'
        ELSE 'Other'
    END AS genre
FROM stream
GROUP BY game
ORDER BY game;


In [ ]:
%%sql
SELECT time
FROM stream
LIMIT 10;


In [ ]:
%%sql
SELECT time, strftime('%S', time) AS seconds
FROM stream
GROUP BY 1
LIMIT 20;


In [ ]:
%%sql
SELECT
    strftime('%H', time) AS hour,
    SUM(viewers) AS total_viewers
FROM stream
WHERE country = 'US'
GROUP BY hour
ORDER BY hour;


In [ ]:
%%sql
SELECT
    s.time,
    s.device_id,
    s.user_id AS stream_user_id,
    c.user_id AS chat_user_id,
    s.channel,
    s.game,
    c.messages
FROM stream s
JOIN chat c ON s.device_id = c.device_id
LIMIT 20;


In [ ]:
%%sql
SELECT
    s.channel,
    COUNT(c.messages) AS total_messages
FROM stream s
JOIN chat c ON s.device_id = c.device_id
WHERE s.game = 'League of Legends'
GROUP BY s.channel
ORDER BY total_messages DESC
LIMIT 10;


In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

# Simulate Twitch `stream` data
stream_data = {
    'game': ['LoL', 'Dota 2', 'CS:GO', 'DayZ', 'HOS', 'Isaac', 'Shows', 'Hearth', 'WoT', 'Agar.io'],
    'viewers': [1070, 472, 302, 239, 210, 171, 170, 90, 86, 71],
    'country': ['US', 'DE', 'US', 'US', 'DE', 'US', 'GB', 'GB', 'CA', 'TR'],
    'time': ['2015-01-01 01:00:00', '2015-01-01 02:00:00', '2015-01-01 03:00:00',
             '2015-01-01 04:00:00', '2015-01-01 05:00:00', '2015-01-01 06:00:00',
             '2015-01-01 07:00:00', '2015-01-01 08:00:00', '2015-01-01 09:00:00',
             '2015-01-01 10:00:00']
}
stream_df = pd.DataFrame(stream_data)

# Simulate Twitch `chat` data
chat_data = {
    'device_id': [1, 2, 3, 4, 5],
    'messages': [10, 15, 20, 5, 8],
    'game': ['LoL', 'CS:GO', 'LoL', 'HOS', 'Dota 2']
}
chat_df = pd.DataFrame(chat_data)

# Create SQLite database
conn = sqlite3.connect("twitch.db")
stream_df.to_sql("stream", conn, index=False, if_exists="replace")
chat_df.to_sql("chat", conn, index=False, if_exists="replace")

# Verify the data loaded
print("Stream Table:")
print(pd.read_sql("SELECT * FROM stream LIMIT 5", conn))

print("\nChat Table:")
print(pd.read_sql("SELECT * FROM chat LIMIT 5", conn))

# Task 1: Bar Graph - Featured Games
games_query = """
SELECT game, SUM(viewers) as total_viewers
FROM stream
GROUP BY game
ORDER BY total_viewers DESC
LIMIT 10
"""
games_data = pd.read_sql(games_query, conn)

# Visualization
plt.figure(figsize=(10, 6))
plt.bar(games_data['game'], games_data['total_viewers'], color='skyblue')
plt.title("Featured Games on Twitch (Viewers)", fontsize=14)
plt.xlabel("Games", fontsize=12)
plt.ylabel("Total Viewers", fontsize=12)
plt.xticks(rotation=45)
plt.show()

# Task 2: Pie Chart - League of Legends Viewers' Whereabouts
lol_query = """
SELECT country, SUM(viewers) as total_viewers
FROM stream
WHERE game = 'LoL'
GROUP BY country
ORDER BY total_viewers DESC
"""
lol_data = pd.read_sql(lol_query, conn)

# Visualization
colors = ['lightskyblue', 'gold', 'lightcoral', 'gainsboro', 'royalblue']
explode = [0.1] + [0] * (len(lol_data) - 1)
plt.figure(figsize=(8, 8))
plt.pie(lol_data['total_viewers'], labels=lol_data['country'], autopct='%1.1f%%',
        explode=explode, colors=colors, shadow=True, startangle=345)
plt.title("League of Legends Viewers' Whereabouts", fontsize=14)
plt.show()

# Task 3: Line Graph - Time Series Analysis
time_query = """
SELECT strftime('%H', time) as hour, SUM(viewers) as total_viewers
FROM stream
GROUP BY hour
ORDER BY hour
"""
time_data = pd.read_sql(time_query, conn)

# Visualization
plt.figure(figsize=(10, 6))
plt.plot(time_data['hour'], time_data['total_viewers'], marker='o', color='skyblue', label="Viewers")
plt.title("Time Series Analysis of Viewers", fontsize=14)
plt.xlabel("Hour (24-Hour Format)", fontsize=12)
plt.ylabel("Total Viewers", fontsize=12)
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# Task 4: Join and Additional Insights
join_query = """
SELECT stream.game, stream.viewers, chat.messages
FROM stream
JOIN chat
ON stream.game = chat.game
"""
join_data = pd.read_sql(join_query, conn)
print("\nStream and Chat Joined Data:")
print(join_data)

# Closing the connection
conn.close()
